In [1]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sqlalchemy import inspect
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from connection import create_db_connection

In [2]:
# Membuat koneksi ke database
engine = create_db_connection()

# Inspect the database to get a list of tables
inspector = inspect(engine)
tables = inspector.get_table_names()

# Check if the connection was successful and if tables were retrieved
if tables:
    print("Connection to the database was successful.")
    print("Tables in the database:", tables)
else:
    print("Failed to connect to the database.")

Connection to the database was successful.
Tables in the database: ['categories', 'failed_jobs', 'migrations', 'password_reset_tokens', 'personal_access_tokens', 'product_images', 'product_reviews', 'products', 'profiles', 'subcategories', 'users']


In [3]:
# Query untuk mengambil data dari tabel products, product_reviews, dan profiles
q_profiles_rating = """
    SELECT 
        pr.id AS review_id, pr.user_id, u.name, pr.product_id, pr.rating, pf.gender,
        pf.age, pf.skin_type_face, pf.hair_issue, pf.skin_type_body, pf.allergy_history,
        pf.preferred_products, pf.avoided_products, pf.specific_needs
    FROM 
        product_reviews pr
    JOIN 
        products p ON pr.product_id = p.id
    JOIN 
        profiles pf ON pr.user_id = pf.user_id
    JOIN 
        users u ON pr.user_id = u.id;
"""

# Mengambil data dari database menggunakan query
dt_profiles_rating_df = pd.read_sql(q_profiles_rating, engine)
display(dt_profiles_rating_df.dtypes)
display(dt_profiles_rating_df)

review_id               int64
user_id                 int64
name                   object
product_id              int64
rating                float64
gender                 object
age                    object
skin_type_face         object
hair_issue             object
skin_type_body         object
allergy_history        object
preferred_products     object
avoided_products       object
specific_needs         object
dtype: object

,review_id,user_id,name,product_id,rating,gender,age,skin_type_face,hair_issue,skin_type_body,allergy_history,preferred_products,avoided_products,specific_needs
0,2,3,Elisa Regina Simanjuntak,44,5.0,Perempuan,18-25,berminyak,berminyak,normal,"[""tidak_ada_alergi""]","[""bahan_alami""]","[""pewangi""]","[""meratakan_warna_kulit"",""menghilangkan_jerawa..."
1,5,3,Elisa Regina Simanjuntak,316,5.0,Perempuan,18-25,berminyak,berminyak,normal,"[""tidak_ada_alergi""]","[""bahan_alami""]","[""pewangi""]","[""meratakan_warna_kulit"",""menghilangkan_jerawa..."
2,14,3,Elisa Regina Simanjuntak,60,5.0,Perempuan,18-25,berminyak,berminyak,normal,"[""tidak_ada_alergi""]","[""bahan_alami""]","[""pewangi""]","[""meratakan_warna_kulit"",""menghilangkan_jerawa..."
3,3,7,Dini Sipahutar,215,5.0,Perempuan,18-25,normal,berminyak,normal,"[""tidak_ada_alergi""]","[""cruelty_free"",""pewangi"",""minyak_mineral""]","[""paraben"",""sls""]","[""meratakan_warna_kulit"",""perlindungan_matahari""]"
4,4,7,Dini Sipahutar,22,5.0,Perempuan,18-25,normal,berminyak,normal,"[""tidak_ada_alergi""]","[""cruelty_free"",""pewangi"",""minyak_mineral""]","[""paraben"",""sls""]","[""meratakan_warna_kulit"",""perlindungan_matahari""]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,434,67,Wilona,294,5.0,Perempuan,18-25,kering,kering,kering,"[""pewangi"",""kandungan_kimia""]","[""bahan_alami""]","[""pewangi""]","[""menghidrasi"",""perlindungan_matahari""]"
433,435,67,Wilona,315,4.0,Perempuan,18-25,kering,kering,kering,"[""pewangi"",""kandungan_kimia""]","[""bahan_alami""]","[""pewangi""]","[""menghidrasi"",""perlindungan_matahari""]"
434,436,67,Wilona,166,4.0,Perempuan,18-25,kering,kering,kering,"[""pewangi"",""kandungan_kimia""]","[""bahan_alami""]","[""pewangi""]","[""menghidrasi"",""perlindungan_matahari""]"
435,437,67,Wilona,168,5.0,Perempuan,18-25,kering,kering,kering,"[""pewangi"",""kandungan_kimia""]","[""bahan_alami""]","[""pewangi""]","[""menghidrasi"",""perlindungan_matahari""]"


In [4]:
# Preprocessing data teks
def preprocess_text(text):
    # Case folding
    text = str(text).lower() 
    # Punctuational removal
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenizing
    tokens = word_tokenize(text)
    # Stop words removal
    stop_words = set(stopwords.words('indonesian'))  # Menggunakan stop words bahasa Indonesia
    tokens = [word for word in tokens if word not in stop_words]
    # Stemming
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    # Menggabungkan kembali token menjadi kalimat
    text = ' '.join(tokens)
    return text

In [5]:
# Preprocessing kolom teks
text_columns = ['gender','skin_type_face', 'hair_issue', 
                'skin_type_body', 'allergy_history', 'preferred_products', 
                'avoided_products', 'specific_needs']
for column in text_columns:
    dt_profiles_rating_df[column] = dt_profiles_rating_df[column].apply(preprocess_text)

display(dt_profiles_rating_df)

,review_id,user_id,name,product_id,rating,gender,age,skin_type_face,hair_issue,skin_type_body,allergy_history,preferred_products,avoided_products,specific_needs
0,2,3,Elisa Regina Simanjuntak,44,5.0,perempuan,18-25,minyak,minyak,normal,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
1,5,3,Elisa Regina Simanjuntak,316,5.0,perempuan,18-25,minyak,minyak,normal,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
2,14,3,Elisa Regina Simanjuntak,60,5.0,perempuan,18-25,minyak,minyak,normal,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
3,3,7,Dini Sipahutar,215,5.0,perempuan,18-25,normal,minyak,normal,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
4,4,7,Dini Sipahutar,22,5.0,perempuan,18-25,normal,minyak,normal,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,434,67,Wilona,294,5.0,perempuan,18-25,kering,kering,kering,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
433,435,67,Wilona,315,4.0,perempuan,18-25,kering,kering,kering,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
434,436,67,Wilona,166,4.0,perempuan,18-25,kering,kering,kering,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
435,437,67,Wilona,168,5.0,perempuan,18-25,kering,kering,kering,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari


In [6]:
print(dt_profiles_rating_df["skin_type_face"].unique())
print(dt_profiles_rating_df["hair_issue"].unique())
print(dt_profiles_rating_df["skin_type_body"].unique())

['minyak' 'normal' 'sensitif' 'none' 'kombinasi' 'kering']
['minyak' 'kering' 'ketombe' 'none' 'normal' 'rontok' 'cabang']
['normal' 'kering' 'minyak' 'none' 'kombinasi']


In [9]:
# convert skin type, hair issue, skin type body to numeric value (int)
def convert_skin_type_face(skin_type): 
    skin_type_dict = {'normal': 0, 'kering': 1, 'minyak': 2, 'sensitif': 3, 'kombinasi': 4}
    return skin_type_dict.get(skin_type, 0)

def convert_hair_issue(hair_issue): 
    hair_issue_dict = {'normal': 1, 'ketombe': 1, 'kering': 2, 'minyak': 3, 'rontok': 4, 'cabang': 5}
    return hair_issue_dict.get(hair_issue, 0)

def convert_skin_type_body(skin_type): 
    skin_type_dict = {'normal': 0, 'kering': 1, 'minyak': 2, 'kombinasi': 3} 
    return skin_type_dict.get(skin_type, 0)

In [10]:
dt_profiles_rating_df["skin_type_face"] = dt_profiles_rating_df["skin_type_face"].apply(convert_skin_type_face) 
dt_profiles_rating_df["hair_issue"] = dt_profiles_rating_df["hair_issue"].apply(convert_hair_issue) 
dt_profiles_rating_df["skin_type_body"] = dt_profiles_rating_df["skin_type_body"].apply(convert_skin_type_body)
display(dt_profiles_rating_df)

,review_id,user_id,name,product_id,rating,gender,age,skin_type_face,hair_issue,skin_type_body,allergy_history,preferred_products,avoided_products,specific_needs
0,2,3,Elisa Regina Simanjuntak,44,5.0,perempuan,18-25,2,3,0,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
1,5,3,Elisa Regina Simanjuntak,316,5.0,perempuan,18-25,2,3,0,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
2,14,3,Elisa Regina Simanjuntak,60,5.0,perempuan,18-25,2,3,0,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
3,3,7,Dini Sipahutar,215,5.0,perempuan,18-25,0,3,0,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
4,4,7,Dini Sipahutar,22,5.0,perempuan,18-25,0,3,0,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,434,67,Wilona,294,5.0,perempuan,18-25,1,2,1,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
433,435,67,Wilona,315,4.0,perempuan,18-25,1,2,1,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
434,436,67,Wilona,166,4.0,perempuan,18-25,1,2,1,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
435,437,67,Wilona,168,5.0,perempuan,18-25,1,2,1,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari


In [11]:
# Fungsi untuk melakukan min-max scaling
def min_max_scaling(column):
    min_value = column.min()
    max_value = column.max()
    return (column - min_value) / (max_value - min_value)

# Melakukan min-max scaling pada kolom
dt_profiles_rating_df["skin_type_face"] = min_max_scaling(dt_profiles_rating_df["skin_type_face"])
dt_profiles_rating_df["hair_issue"] = min_max_scaling(dt_profiles_rating_df["hair_issue"])
dt_profiles_rating_df["skin_type_body"] = min_max_scaling(dt_profiles_rating_df["skin_type_body"])

#  Membuat nilai numerik 2 angka dibelakang koma
dt_profiles_rating_df = dt_profiles_rating_df.round(2)

# Menampilkan data
display(dt_profiles_rating_df)

,review_id,user_id,name,product_id,rating,gender,age,skin_type_face,hair_issue,skin_type_body,allergy_history,preferred_products,avoided_products,specific_needs
0,2,3,Elisa Regina Simanjuntak,44,5.0,perempuan,18-25,0.50,0.6,0.00,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
1,5,3,Elisa Regina Simanjuntak,316,5.0,perempuan,18-25,0.50,0.6,0.00,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
2,14,3,Elisa Regina Simanjuntak,60,5.0,perempuan,18-25,0.50,0.6,0.00,tidak ada alergi,bahan alami,wangi,rata warna kulitmenghilangkan jerawatmenghidra...
3,3,7,Dini Sipahutar,215,5.0,perempuan,18-25,0.00,0.6,0.00,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
4,4,7,Dini Sipahutar,22,5.0,perempuan,18-25,0.00,0.6,0.00,tidak ada alergi,cruelty freepewangiminyak mineral,parabensls,rata warna kulitperlindungan matahari
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,434,67,Wilona,294,5.0,perempuan,18-25,0.25,0.4,0.33,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
433,435,67,Wilona,315,4.0,perempuan,18-25,0.25,0.4,0.33,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
434,436,67,Wilona,166,4.0,perempuan,18-25,0.25,0.4,0.33,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari
435,437,67,Wilona,168,5.0,perempuan,18-25,0.25,0.4,0.33,pewangikandungan kimia,bahan alami,wangi,menghidrasiperlindungan matahari


In [12]:
# Hitung rata rata dari user yang ada
user_vectors = dt_profiles_rating_df.groupby('user_id')[['skin_type_face', 'hair_issue', 'skin_type_body']].mean().round(2)

# Rset index matriks
user_vectors.reset_index(inplace=True)
display(user_vectors)

,user_id,skin_type_face,hair_issue,skin_type_body
0,3,0.50,0.6,0.00
1,7,0.00,0.6,0.00
2,8,0.75,0.4,0.33
3,11,0.00,0.2,0.67
4,12,0.00,0.0,0.00
5,13,0.50,0.2,0.00
6,14,0.50,0.2,0.00
7,15,0.50,0.4,0.00
8,16,0.00,0.4,0.00
9,17,0.00,0.2,0.00


In [13]:
# Filter hanya data user_id yang ada dalam file CSV
user_vectors = user_vectors[user_vectors['user_id'].isin(dt_profiles_rating_df['user_id'].unique())]

# Simpan user_id sebelum dihapus dari user_vectors
user_ids = user_vectors['user_id']

# Hapus kolom user_id sebelum menghitung similaritas
user_vectors = user_vectors.drop('user_id', axis=1)

# Hitung similaritas kosinus antara semua pasangan pengguna
user_similarities = cosine_similarity(user_vectors)

# Konversi matriks similaritas ke dataframe pandas
user_similarities = pd.DataFrame(user_similarities, index=user_ids, columns=user_ids).round(2)
display(user_similarities)

user_id,3,7,8,11,12,13,14,15,16,17,...,58,59,60,61,62,63,64,65,66,67
user_id,,,,,,,,,,,,,,,,,,,,,
3,1.00,0.77,0.86,0.22,0.0,0.88,0.88,0.98,0.77,0.77,...,0.99,1.00,0.83,0.42,0.98,0.77,0.96,0.84,0.71,0.81
7,0.77,1.00,0.44,0.29,0.0,0.37,0.37,0.62,1.00,1.00,...,0.85,0.77,0.77,0.18,0.62,1.00,0.70,0.36,0.92,0.69
8,0.86,0.44,1.00,0.47,0.0,0.93,0.93,0.92,0.44,0.44,...,0.81,0.86,0.84,0.76,0.92,0.44,0.95,0.99,0.54,0.87
11,0.22,0.29,0.47,1.00,0.0,0.11,0.11,0.18,0.29,0.29,...,0.24,0.22,0.71,0.89,0.18,0.29,0.48,0.38,0.63,0.75
12,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.88,0.37,0.93,0.11,0.0,1.00,1.00,0.96,0.37,0.37,...,0.81,0.88,0.64,0.47,0.96,0.37,0.87,0.96,0.34,0.66
14,0.88,0.37,0.93,0.11,0.0,1.00,1.00,0.96,0.37,0.37,...,0.81,0.88,0.64,0.47,0.96,0.37,0.87,0.96,0.34,0.66
15,0.98,0.62,0.92,0.18,0.0,0.96,0.96,1.00,0.62,0.62,...,0.94,0.98,0.78,0.45,1.00,0.62,0.95,0.92,0.58,0.77
16,0.77,1.00,0.44,0.29,0.0,0.37,0.37,0.62,1.00,1.00,...,0.85,0.77,0.77,0.18,0.62,1.00,0.70,0.36,0.92,0.69


In [14]:
# Ambil semua data produk berdasarkan id
items = dt_profiles_rating_df['product_id'].unique()

# Ambil semuda data user berdasarkan id sebelum dilakukan proses data
unique_user_ids = dt_profiles_rating_df['user_id'].unique()
display(items)

array([ 44, 316,  60, 215,  22, 311, 286,  82, 140, 112,  63,  51,  21,
        18,   5,   8,  68,  92,  85, 216, 307, 280, 276, 236,  58,  24,
       300, 315,   7, 294, 134,  42,  43, 123, 167, 168, 199,  84, 283,
        16, 309,  47,  20, 263,  35, 135, 219,  61,  78,  93,  62,  39,
       282, 165, 169, 261, 191,   6, 295, 122, 314,  46, 312, 306,  59,
       188,  36, 138, 298,  81, 166, 310,  56,  23, 161, 207, 252,  88,
        32, 142,  40, 179,  95, 141, 163, 170, 120, 208,  94, 302, 228,
       193, 147,   4, 171,  69,  79,  87, 114, 274, 260, 160,   9, 304,
       234, 218, 285, 299,  14,  15,  37,  25,  83, 121, 156, 159,  66,
       164,  57, 211,  13, 305,  50, 116, 149,  33,  12, 175, 130, 273,
       217,  30,  48, 313, 259, 137,  73, 232,  45,  65,  75,  34, 187,
        67, 265, 255,  28, 233, 214,  64,  74,  96, 278, 267, 258, 195,
       229, 292, 254, 251, 227, 178, 139,  99,  98,  97,  71,  55,  26,
       146, 262], dtype=int64)

In [15]:
# Initialize a dictionary to store the recommendation values for each user
recommendations = {}

# Iterate over each user ID from the preprocessed data
for user_id in unique_user_ids:
    # Calculate the sum of similarities for the current user
    similarity_sum = user_similarities.loc[user_id].sum()
    predictions = {}

    if similarity_sum > 0:  # Check if similarity_sum is greater than 0 to avoid division by zero
        for item in items:
            # Filter ratings for the item from other users
            other_user_ratings = dt_profiles_rating_df[dt_profiles_rating_df['product_id'] == item]

            # Calculate the sum of ratings weighted by similarity values
            rating_sum = 0
            for other_user_id in other_user_ratings['user_id']:
                if other_user_id != user_id:
                    rating = other_user_ratings[other_user_ratings['user_id'] == other_user_id]['rating'].values[0]
                    similarity = user_similarities.loc[user_id, other_user_id]
                    rating_sum += rating * similarity

            # Calculate the predicted rating for the item
            predictions[item] = rating_sum / similarity_sum

    # Store the predictions for the current user
    recommendations[user_id] = predictions

# Convert the recommendations dictionary to a pandas dataframe
recommendations_df = pd.DataFrame(recommendations)
recommendations_df = recommendations_df.T
recommendations_df = recommendations_df.round(2)

# Fill NaN values and 0 values with 0
recommendations_df = recommendations_df.fillna(0).replace(0, 0)

display(recommendations_df)

,44,316,60,215,22,311,286,82,140,112,...,178,139,99,98,97,71,55,26,146,262
3,0.24,0.73,0.36,0.63,0.09,0.63,0.24,1.29,0.50,0.09,...,0.07,0.08,0.07,0.07,0.14,0.05,0.03,0.08,0.07,0.07
7,0.34,0.69,0.41,0.56,0.00,0.70,0.13,1.26,0.28,0.00,...,0.11,0.14,0.11,0.11,0.19,0.08,0.05,0.14,0.08,0.08
8,0.35,0.95,0.50,0.54,0.05,0.52,0.20,1.21,0.52,0.05,...,0.05,0.06,0.05,0.05,0.13,0.04,0.03,0.06,0.08,0.08
11,0.33,0.94,0.42,0.42,0.07,0.64,0.13,1.13,0.34,0.07,...,0.12,0.15,0.12,0.12,0.26,0.09,0.06,0.15,0.14,0.14
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.36,0.95,0.52,0.57,0.05,0.49,0.22,1.22,0.57,0.05,...,0.04,0.04,0.04,0.04,0.10,0.03,0.02,0.04,0.07,0.07
14,0.36,0.95,0.52,0.57,0.05,0.49,0.22,1.09,0.46,0.05,...,0.04,0.04,0.04,0.04,0.10,0.03,0.02,0.04,0.07,0.07
15,0.36,0.79,0.49,0.49,0.07,0.58,0.23,1.26,0.53,0.07,...,0.05,0.07,0.05,0.05,0.13,0.04,0.03,0.07,0.07,0.07
16,0.34,0.69,0.41,0.71,0.15,0.85,0.28,1.26,0.40,0.15,...,0.11,0.14,0.11,0.11,0.19,0.08,0.05,0.14,0.08,0.08
17,0.34,0.69,0.41,0.71,0.15,0.85,0.28,1.38,0.40,0.15,...,0.11,0.14,0.11,0.11,0.19,0.08,0.05,0.14,0.08,0.08


In [16]:
# Hitung MAE
absolute_errors = []
for user_id in unique_user_ids:
    user_actual_ratings = dt_profiles_rating_df.loc[dt_profiles_rating_df['user_id'] == user_id, ['product_id', 'rating']]
    user_actual_ratings = user_actual_ratings.dropna(subset=['product_id', 'rating'])  # Hapus baris dengan data kosong
    user_actual_ratings = user_actual_ratings.set_index('product_id')['rating'].to_dict()

    user_predicted_ratings = recommendations_df.loc[user_id].to_dict()

    for product_id in user_actual_ratings.keys():
        if product_id in user_predicted_ratings:
            actual_rating = user_actual_ratings[product_id]
            predicted_rating = user_predicted_ratings[product_id]
            absolute_error = abs(actual_rating - predicted_rating)
            absolute_errors.append(absolute_error)

# Hitung MAE hanya jika ada nilai dalam absolute_errors
if absolute_errors:
    mae = np.mean(absolute_errors)
    print(f"Mean Absolute Error (MAE): {mae:.3f}")
else:
    print("Tidak ada data untuk menghitung MAE.")

Mean Absolute Error (MAE): 4.177


In [17]:
# Iterate over each user ID
for user_id in unique_user_ids:
    # Look up the user's name in the data_df dataframe
    user_name = dt_profiles_rating_df.loc[dt_profiles_rating_df['user_id'] == user_id, 'name'].values[0]

    # Select the row for the specified user
    user_recs = recommendations_df.loc[user_id]

    # Sort the series in descending order and select the top 10 values
    top_recs = user_recs.sort_values(ascending=False).iloc[:10].index.tolist()

    # Print the recommended product IDs and names
    print("Top 10 recommended products for user_id = {} : ({})".format(user_id, user_name))
    print(f"Product IDs: {top_recs}")
    print()

Top 10 recommended products for user_id = 3 : (Elisa Regina Simanjuntak)
Product IDs: [82, 58, 316, 215, 311, 68, 42, 168, 135, 140]

Top 10 recommended products for user_id = 7 : (Dini Sipahutar)
Product IDs: [82, 58, 311, 316, 215, 168, 42, 135, 57, 47]

Top 10 recommended products for user_id = 8 : (Gladys)
Product IDs: [82, 58, 316, 62, 42, 168, 68, 215, 311, 140]

Top 10 recommended products for user_id = 11 : (Suandika Napitupulu)
Product IDs: [58, 82, 316, 168, 62, 311, 47, 42, 46, 135]

Top 10 recommended products for user_id = 12 : (Elisa)
Product IDs: [44, 164, 15, 37, 25, 83, 121, 156, 159, 66]

Top 10 recommended products for user_id = 13 : (Emy Sonia Sinambela)
Product IDs: [82, 58, 316, 42, 62, 68, 215, 140, 309, 168]

Top 10 recommended products for user_id = 14 : (Stefhani Kezia)
Product IDs: [58, 82, 316, 42, 62, 68, 215, 168, 135, 60]

Top 10 recommended products for user_id = 15 : (Josep Phyto Napitupulu)
Product IDs: [82, 58, 316, 311, 42, 168, 140, 309, 215, 60]

T